In [31]:
print("Hello LangChain")

Hello LangChain


In [32]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

#load_dotenv(dotenv_path='.env')
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_E


In [33]:
# prompt + llm + output 

# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("user", "{input}") ]
)
print(type(prompt))
print(prompt)

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [34]:
prompt_text = prompt.format(input="파이썬은 무엇인가요? 자세하게 설명해주세요")
print(type(prompt_text))
print(prompt_text)

<class 'str'>
System: 당신은 개발자입니다.
Human: 파이썬은 무엇인가요? 자세하게 설명해주세요


In [35]:
#llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-0125")

# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="moonshotai/kimi-k2-instruct-0905",
    model="openai/gpt-oss-120b",
    temperature=0.7
)
print(type(llm))
print(llm)

<class 'langchain_openai.chat_models.base.ChatOpenAI'>
client=<openai.resources.chat.completions.completions.Completions object at 0x00000216AC1EB6B0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000216AC1EB9B0> root_client=<openai.OpenAI object at 0x00000216AC1EB530> root_async_client=<openai.AsyncOpenAI object at 0x00000216AC1FE0C0> model_name='openai/gpt-oss-120b' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [36]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
응답: ## 파이썬(Python)이란?

파이썬은 **고수준(high‑level), 인터프리터 방식, 동적 타이핑(dynamic typing)** 을 지원하는 프로그래밍 언어입니다. 1991년 네덜란드의 프로그래머 **귀도 반 로썸(Guido van Rossum)** 이 처음 발표했으며, 현재는 전 세계 수많은 개발자와 기업이 널리 사용하고 있습니다.  

---

## 1. 파이썬의 주요 특징

| 특징 | 설명 | 장점 |
|------|------|------|
| **읽기 쉬운 문법** | 들여쓰기(Indentation)로 블록을 구분하고, 불필요한 기호(세미콜론, 중괄호 등)가 거의 없음 | 코드 가독성·유지보수성이 뛰어남 |
| **동적 타이핑** | 변수 선언 시 타입을 명시하지 않음 (`x = 10` → 정수, `x = "hello"` → 문자열) | 빠른 프로토타이핑·코드량 감소 |
| **인터프리터 언어** | 소스 코드를 바로 실행(컴파일 단계가 없음) | 즉시 피드백·디버깅이 용이 |
| **다중 패러다임** | 절차적, 객체지향(OOP), 함수형 프로그래밍 지원 | 다양한 문제 해결 방식 선택 가능 |
| **풍부한 표준 라이브러리** | 파일 I/O, 네트워킹, 웹, 데이터베이스, 압축 등 2천 개 이상의 모듈 포함 | 외부 패키지 없이도 많은 작업 수행 가능 |
| **광범위한 서드파티 생태계** | `pip`(패키지 매니저)를 통해 PyPI(Python Package Index)에서 300,000+ 패키지 설치 가능 | 과학·데이터, 웹, 머신러닝, 자동화 등 거의 모든 분야에 적용 |
| **플랫폼 독립성** | Windows, macOS, Linux, BSD, 심지어 모바일(iOS/Android)까지 지원 | 한 번 작성하면 다양한 환경에서 실행 가능 |
| **대규모 커뮤니티** | 공식 문서, 튜토리얼, Stack Overflow, GitHu

LCEL

In [37]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [38]:
chain = prompt | llm | output_parser
print(type(chain))
print(chain)

<class 'langchain_core.runnables.base.RunnableSequence'>
first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]) middle=[ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000216AC1EB6B0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000216AC1EB9B0>, root_client=<openai.OpenAI object at 0x00000216AC1EB530>, root_async_client=<openai.AsyncOpenAI object at 0x00000216AC1FE0C0>, model_name='openai/gpt-oss-120b', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.groq.com/openai/v1')] last=StrOutputPars

In [29]:
response = chain.invoke({"input":"LangChain은 무엇인가요?"})

In [30]:
print(type(response))
print(response)

<class 'str'>
**LangChain**은 **대규모 언어 모델(LLM)** 을 활용한 **애플리케이션**을 보다 쉽고 효율적으로 개발할 수 있게 도와주는 **오픈소스 프레임워크**입니다.  
주요 목표는 “LLM을 단순히 질문‑답변 엔진으로 쓰는 수준을 넘어, **데이터 연동·프롬프트 관리·워크플로우 제어**까지 포함한 **복합적인 애플리케이션**을 만들 수 있게 하는 것”입니다.

---

## 1️⃣ 핵심 개념

| 요소 | 설명 | 주요 역할 |
|------|------|-----------|
| **Chains** | 여러 **Component**(LLM, Prompt, Tool 등)를 순차·조건부로 연결한 흐름 | 프롬프트 생성 → LLM 호출 → 결과 후처리 등 복잡한 로직을 한 번에 정의 |
| **Agents** | **동적**으로 **Tool**(검색 API, DB, 계산기 등)을 선택·호출하는 **자율적인** 엔티티 | “날씨를 알려줘” → 검색 → LLM에게 요약 요청 등 상황에 맞게 도구를 사용 |
| **Prompts** | 템플릿 기반의 **프롬프트 관리**·**동적 변수 삽입** | 재사용 가능한 프롬프트, 체인별 파라미터 관리 |
| **Memory** | 대화·작업 흐름에서 **상태(컨텍스트)를 유지** | 챗봇에서 이전 대화를 기억하거나, 단계별 결과를 누적 |
| **Indexes / Retrieval** | 문서·데이터베이스를 **벡터화·검색**하여 LLM에 **컨텍스트** 제공 | RAG(Retrieval‑Augmented Generation) 파이프라인 구현 |
| **Tools** | 외부 서비스·함수·API 등 **LLM이 호출할 수 있는 기능** | 검색 엔진, 데이터베이스, 계산기, 웹 스크래핑 등 |
| **Callbacks** | 실행 흐름에 **후킹**(로그, 메트릭, 트레이싱) | 디버깅·성능 모니터링·사용자 피드백 수집 |

---

## 2️⃣ 주요 구성 요소 (Python 예시)

```p